In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from django.db.models import Sum, Max, Subquery, Q, F, Avg, Count, Min, Max, Sum, F, Q
from django.db.models import Value, IntegerField, When, FloatField
from django.db.models.functions import Coalesce, Greatest, Floor
from account.models import User
from booth.models import Booth, Transaction, Participation
from player.models import Player
import random
import pandas as pd


In [2]:
p = User.objects.get(id=101).player

In [3]:
p.get_scores()

{'health_score': 40.0,
 'skill_score': 60.0,
 'growth_score': 5.0,
 'relationship_score': 50.0,
 'cash': 15200.0,
 'academic_level': '小學畢業',
 'steps': 15.0,
 'bank_amount': 0.0,
 'pay': 100.0,
 'receive': 200.0,
 'deposit': 500.0,
 'withdrawal': 500.0}

In [4]:
# Define the columns
columns = ['a', 'b', 'c']

# Use a lambda function to create the aggregation dictionary
aggregation_funcs = lambda cols: {col: 'sum' if col in ['a', 'b'] else 'max' for col in cols}

# Generate the result
result = aggregation_funcs(columns)

# Print the result
print(result)

{'a': 'sum', 'b': 'sum', 'c': 'max'}


In [44]:
pay = Case(
    When(type='pay', then='money'), 
    default=Value(0.0),
    output_field = FloatField()
)
receive = Case(
    When(type='receive', then='money'), 
    default=Value(0.0),
    output_field = FloatField()
)
deposit = Case(
    When(type='deposit', then='money'), 
    default=Value(0.0),
    output_field = FloatField()
)
withdrawal = Case(
    When(type='withdrawal', then='money'), 
    default=Value(0.0),
    output_field = FloatField()
)
withdrawal_with_interest = Case(
    When(type='withdrawal', then=F('money') * (Value(1.0) + F('interest_rate'))), 
    default=Value(0.0),
    output_field = FloatField()
)
transactions = self.transaction_player.aggregate(
    health_score=Sum(Value(0.0)),
    skill_score=Sum(Value(0.0)),
    growth_score=Sum(Value(0.0)),
    relationship_score=Sum(Value(0.0)),
    cash=Sum(receive + withdrawal_with_interest - pay - deposit),
    bank_amount=Sum(deposit-withdrawal),
    academic_level=Sum(Value(0.0)),
    steps=Sum(Value(0.0)),
    pay=Sum(pay),
    receive=Sum(receive),
    deposit=Sum(deposit),
    withdrawal=Sum(withdrawal)
)

In [40]:
participations = self.participation_player.aggregate(
    health_score=Coalesce(Sum('health_score'), Value(0)),
    skill_score=Coalesce(Sum('skill_score'), Value(0)),
    growth_score=Coalesce(Sum('growth_score'), Value(0)),
    relationship_score=Coalesce(Sum('relationship_score'), Value(0)),
    cash=Coalesce(Sum('money'), Value(0)),
    academic_level=Coalesce(Max('academic_level'), Value(0)),
    steps=Coalesce(Sum('steps'), Value(0)),
)

In [47]:
pd.DataFrame([transactions, participations]).sum()

health_score            0.0
skill_score             0.0
growth_score            0.0
relationship_score      0.0
cash                  200.0
bank_amount             0.0
academic_level          0.0
steps                   0.0
pay                   100.0
receive               200.0
deposit               500.0
withdrawal            500.0
dtype: float64